## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Input, Conv2D, Activation, AveragePooling2D
from keras.regularizers import l2
from keras.datasets import cifar10
from keras.models import Model
import keras
import numpy as np
import tensorflow as tf
import os
#from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
# x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
# one_hot=OneHotEncoder()
# y_train=one_hot.fit_transform(y_train).toarray()
# y_test=one_hot.transform(y_test).toarray()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


model = resnet_v2(input_shape=(32,32,3), depth=18 * 9 + 2)

In [5]:
from keras.optimizers import Adam

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

Learning rate:  0.001

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_____________________________________________________________________

__________________________________________________________________________________________________
batch_normalization_82 (BatchNo (None, 16, 16, 128)  512         add_27[0][0]                     
__________________________________________________________________________________________________
activation_82 (Activation)      (None, 16, 16, 128)  0           batch_normalization_82[0][0]     
__________________________________________________________________________________________________
conv2d_85 (Conv2D)              (None, 16, 16, 64)   8256        activation_82[0][0]              
__________________________________________________________________________________________________
batch_normalization_83 (BatchNo (None, 16, 16, 64)   256         conv2d_85[0][0]                  
__________________________________________________________________________________________________
activation_83 (Activation)      (None, 16, 16, 64)   0           batch_normalization_83[0][0]     
__________

In [6]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler
# Prepare callbacks for model saving and for learning rate adjustment.

checkpoint = ModelCheckpoint(filepath='./logs/cifar10_ResNetv2-164_model_'+ 'ep{epoch:03d}-acc{acc:.3f}-val_acc{val_acc:.3f}.h5',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule,
                                     verbose=1)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=1e-7,
                               verbose=1)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [7]:
from imgaug import augmenters as iaa

augmentation = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    # Strengthen or weaken the contrast in each image.
    iaa.Sometimes(0.5,  iaa.GammaContrast((0.9, 1.1), per_channel=0.1))
#     iaa.MultiplySaturation((0.5, 1.5)),
#     iaa.AddToHue((-50, 50)),
#     iaa.GammaContrast((0.9, 1.1), per_channel=0.1),
#     iaa.Multiply((0.5, 1.5)),
#     iaa.CoarseDropout([0.05], size_percent=(0.001, 0.01))
    ], random_order=True) # apply augmenters in random order
#augmentation = augmentation.to_deterministic()

In [8]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.figure(figsize=(50,50))
# for i in range(81):
#     plt.subplot(9,9,i+1)  
#     plt.axis('off')
#     plt.imshow(train_generator.next()[0][i])

In [9]:
from keras.preprocessing.image import ImageDataGenerator

# preprocessing image and divide validaiton set
train_datagen=ImageDataGenerator(preprocessing_function=augmentation.augment_image,
                                 rescale=1./255,
                                 zca_epsilon=1e-06,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 fill_mode='nearest',
                                 horizontal_flip=True,)

test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow(x_train, 
                                   y_train,
                                   batch_size=128,
                                   shuffle=True)


test_generator = test_datagen.flow(x_test, 
                                   y_test,
                                   batch_size=128)

In [10]:
model.fit_generator(generator=train_generator,
                    validation_data=test_generator,
                    epochs=200, verbose=1, workers=4,
                    callbacks=callbacks)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/200
Learning rate:  0.001

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 153s 391ms/step - loss: 3.4579 - acc: 0.4476 - val_loss: 3.0268 - val_acc: 0.3945

Epoch 00001: val_acc improved from -inf to 0.39450, saving model to ./logs/cifar10_ResNetv2-164_model_ep001-acc0.448-val_acc0.395.h5
Epoch 2/200
Learning rate:  0.001

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 105s 269ms/step - loss: 2.0924 - acc: 0.6002 - val_loss: 2.1952 - val_acc: 0.5464

Epoch 00002: val_acc improved from 0.39450 to 0.54640, saving model to ./logs/cifar10_ResNetv2-164_model_ep002-acc0.600-val_acc0.546.h5
Epoch 3/200
Learning rate:  0.001

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 106s 272ms/step - loss: 1.6622 - acc

391/391 [==============================] - 106s 272ms/step - loss: 0.6091 - acc: 0.8882 - val_loss: 1.0499 - val_acc: 0.7742

Epoch 00026: val_acc did not improve from 0.82710
Epoch 27/200
Learning rate:  0.001

Epoch 00027: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 107s 274ms/step - loss: 0.5976 - acc: 0.8909 - val_loss: 0.7404 - val_acc: 0.8475

Epoch 00027: val_acc improved from 0.82710 to 0.84750, saving model to ./logs/cifar10_ResNetv2-164_model_ep027-acc0.891-val_acc0.848.h5
Epoch 28/200
Learning rate:  0.001

Epoch 00028: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 107s 274ms/step - loss: 0.5856 - acc: 0.8956 - val_loss: 0.9291 - val_acc: 0.8027

Epoch 00028: val_acc did not improve from 0.84750
Epoch 29/200
Learning rate:  0.001

Epoch 00029: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 107s 272ms/step - loss: 0.5772 - a

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.285041). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


391/391 [==============================] - 106s 271ms/step - loss: 0.4706 - acc: 0.9176 - val_loss: 0.7565 - val_acc: 0.8440

Epoch 00046: val_acc did not improve from 0.86870
Epoch 47/200
Learning rate:  0.001

Epoch 00047: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 106s 271ms/step - loss: 0.4628 - acc: 0.9215 - val_loss: 0.6802 - val_acc: 0.8592

Epoch 00047: val_acc did not improve from 0.86870
Epoch 48/200
Learning rate:  0.001

Epoch 00048: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 106s 270ms/step - loss: 0.4605 - acc: 0.9217 - val_loss: 0.7813 - val_acc: 0.8373

Epoch 00048: val_acc did not improve from 0.86870
Epoch 49/200
Learning rate:  0.001

Epoch 00049: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 105s 270ms/step - loss: 0.4536 - acc: 0.9238 - val_loss: 0.7959 - val_acc: 0.8333

Epoch 00049: val_acc did not improve 

391/391 [==============================] - 105s 270ms/step - loss: 0.3875 - acc: 0.9382 - val_loss: 0.6649 - val_acc: 0.8638

Epoch 00074: val_acc did not improve from 0.88480

Epoch 00074: ReduceLROnPlateau reducing learning rate to 0.00031622778103685084.
Epoch 75/200
Learning rate:  0.001

Epoch 00075: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 106s 270ms/step - loss: 0.3877 - acc: 0.9382 - val_loss: 0.6237 - val_acc: 0.8779

Epoch 00075: val_acc did not improve from 0.88480
Epoch 76/200
Learning rate:  0.001

Epoch 00076: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 106s 270ms/step - loss: 0.3855 - acc: 0.9377 - val_loss: 0.6519 - val_acc: 0.8673

Epoch 00076: val_acc did not improve from 0.88480
Epoch 77/200
Learning rate:  0.001

Epoch 00077: LearningRateScheduler setting learning rate to 0.001.
391/391 [==============================] - 105s 269ms/step - loss: 0.3848 - acc: 

391/391 [==============================] - 105s 269ms/step - loss: 0.1940 - acc: 0.9911 - val_loss: 0.4324 - val_acc: 0.9305

Epoch 00101: val_acc did not improve from 0.93510
Epoch 102/200
Learning rate:  0.0001

Epoch 00102: LearningRateScheduler setting learning rate to 0.0001.
391/391 [==============================] - 105s 269ms/step - loss: 0.1909 - acc: 0.9915 - val_loss: 0.4206 - val_acc: 0.9308

Epoch 00102: val_acc did not improve from 0.93510
Epoch 103/200
Learning rate:  0.0001

Epoch 00103: LearningRateScheduler setting learning rate to 0.0001.
391/391 [==============================] - 105s 269ms/step - loss: 0.1903 - acc: 0.9910 - val_loss: 0.4431 - val_acc: 0.9283

Epoch 00103: val_acc did not improve from 0.93510
Epoch 104/200
Learning rate:  0.0001

Epoch 00104: LearningRateScheduler setting learning rate to 0.0001.
391/391 [==============================] - 106s 270ms/step - loss: 0.1880 - acc: 0.9911 - val_loss: 0.4287 - val_acc: 0.9315

Epoch 00104: val_acc did not

Epoch 128/200
Learning rate:  1e-05

Epoch 00128: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/step - loss: 0.1494 - acc: 0.9962 - val_loss: 0.4031 - val_acc: 0.9366

Epoch 00128: val_acc did not improve from 0.93750
Epoch 129/200
Learning rate:  1e-05

Epoch 00129: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 270ms/step - loss: 0.1491 - acc: 0.9966 - val_loss: 0.4039 - val_acc: 0.9364

Epoch 00129: val_acc did not improve from 0.93750
Epoch 130/200
Learning rate:  1e-05

Epoch 00130: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/step - loss: 0.1475 - acc: 0.9970 - val_loss: 0.4015 - val_acc: 0.9372

Epoch 00130: val_acc did not improve from 0.93750
Epoch 131/200
Learning rate:  1e-05

Epoch 00131: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/st


Epoch 00155: val_acc did not improve from 0.93850

Epoch 00155: ReduceLROnPlateau reducing learning rate to 3.1622775802825263e-06.
Epoch 156/200
Learning rate:  1e-05

Epoch 00156: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/step - loss: 0.1420 - acc: 0.9971 - val_loss: 0.4022 - val_acc: 0.9369

Epoch 00156: val_acc did not improve from 0.93850
Epoch 157/200
Learning rate:  1e-05

Epoch 00157: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/step - loss: 0.1414 - acc: 0.9972 - val_loss: 0.4018 - val_acc: 0.9374

Epoch 00157: val_acc did not improve from 0.93850
Epoch 158/200
Learning rate:  1e-05

Epoch 00158: LearningRateScheduler setting learning rate to 1e-05.
391/391 [==============================] - 105s 269ms/step - loss: 0.1400 - acc: 0.9977 - val_loss: 0.4038 - val_acc: 0.9376

Epoch 00158: val_acc did not improve from 0.93850
Epoch 159/200
Learning rate

391/391 [==============================] - 105s 269ms/step - loss: 0.1390 - acc: 0.9979 - val_loss: 0.4026 - val_acc: 0.9376

Epoch 00183: val_acc did not improve from 0.93850
Epoch 184/200
Learning rate:  5e-07

Epoch 00184: LearningRateScheduler setting learning rate to 5e-07.
391/391 [==============================] - 105s 270ms/step - loss: 0.1400 - acc: 0.9975 - val_loss: 0.4018 - val_acc: 0.9381

Epoch 00184: val_acc did not improve from 0.93850
Epoch 185/200
Learning rate:  5e-07

Epoch 00185: LearningRateScheduler setting learning rate to 5e-07.
391/391 [==============================] - 105s 269ms/step - loss: 0.1390 - acc: 0.9977 - val_loss: 0.4027 - val_acc: 0.9379

Epoch 00185: val_acc did not improve from 0.93850
Epoch 186/200
Learning rate:  5e-07

Epoch 00186: LearningRateScheduler setting learning rate to 5e-07.
391/391 [==============================] - 105s 269ms/step - loss: 0.1394 - acc: 0.9972 - val_loss: 0.4029 - val_acc: 0.9381

Epoch 00186: val_acc did not impro

In [11]:

# classifier=Sequential()

# #卷積組合
# classifier.add(Convolution2D('自己設計參數'))#32,3,3,input_shape=(32,32,3),activation='relu''
# classifier.add(BatchNormalization())

# '''自己決定MaxPooling2D放在哪裡'''
# #classifier.add(MaxPooling2D(pool_size=(2,2)))

# #卷積組合
# classifier.add(Convolution2D('自己設計參數'))
# classifier.add(BatchNormalization())

# #flatten
# classifier.add(Flatten())

# #FC
# classifier.add(Dense('自己設計FC層參數')) #output_dim=100,activation=relu

# #輸出
# classifier.add(Dense(output_dim=10,activation='輸出函數應該用什麼？'))

# #超過兩個就要選categorical_crossentrophy
# classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# classifier.fit(x_train,y_train,batch_size=100,epochs=100)

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [16]:
# input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
input_example=(np.zeros(shape=(1,32,32,3))* (1./255))
model.predict(input_example)

array([[0.31479806, 0.00427027, 0.11111799, 0.04385268, 0.2517599 ,
        0.00138484, 0.03978383, 0.13027717, 0.09946913, 0.00328611]],
      dtype=float32)